# Peer-Graded Assignment : Segmenting and Clustering Neighborhoods in Toronto

## Kenny Lu

In this assignment, I will explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so I need to learn to be agile and refine the skill to learn new libraries and tools.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. I will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, I will replicate the analysis that did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## Step 1: Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.

### 1. Import libraries

Lets first import the required libraries.
We will need requests for getting the HTML contents of the website and store the data on a Pandas Dataframe.

In [4]:
import numpy as np                                                            # library to handle data in a vectorized manner
import pandas as pd                                                           # library for data analsysis

import requests                                                               # library to handle requests
from bs4 import BeautifulSoup                                                 # library to parse HTML and XML documents

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)



### 2. crape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe

#### a) send the GET request to retrieve List of postal codes of Canada from Wikipedia

In [5]:
# Create a handle, page, to handle the contents of the website
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)


#### b) obtain the data that is in the table of postal codes

In [6]:
# parse data from Wikipedia into a beautifulsoup object
soup = BeautifulSoup(page.content,'html.parser')

# Find the wikitable with needed information and store it to  wiki_table
wiki_table=soup.find("table",{"class":"wikitable sortable"})

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []
 
# append the data into the respective lists
for row in wiki_table.find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))                # postal code, rstrip('\n') remove new lines in neighborhood cell
        boroughList.append(cells[1].text.rstrip('\n'))                   # borough, rstrip('\n') remove new lines in neighborhood cell
        neighborhoodList.append(cells[2].text.rstrip('\n'))              # neighborhood, rstrip('\n') remove new lines in neighborhood cell

#### c) transform the data into a pandas dataframe
        The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### The number of rows of your dataframe is

In [8]:
toronto_df.shape

(180, 3)

#### d) clear the pandas dataframe 
<OL>
<li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. </li>
<li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
<li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
</OL>

##### Drop cells with a borough that is "Not assigned"

In [9]:
# Drop cells with a borough that is "Not assigned"
toronto_df_dropna = toronto_df[toronto_df["Borough"] !=  'Not assigned']
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### Group neighborhood in the same borough with a comma

In [10]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### for Neighborhood="Not assigned", make the value the same as Borough

In [11]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### The number of rows of the cleaned dataframe is

In [12]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

#### Check whether it is the same as required by the question

In [13]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
